In [29]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [30]:
# association recognition - cued recall
with open("../Data/simuS2_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 300]
# df_test = df_test.loc[df_test.session < 300]

In [31]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session,pair_idx
0,1622,811,WORLD,LENS,0,8
1,897,346,MINK,COLLEGE,0,19
2,569,828,FIREMAN,LIP,0,0
3,1077,674,PIT,GYM,0,21
4,1422,1103,SWITCH,POOL,0,13
...,...,...,...,...,...,...
143995,98,1244,BATH,SCAPEGOAT,999,3
143996,638,1579,GLACIER,WAIST,999,1
143997,328,532,CLOUD,EMPLOYER,999,20
143998,489,1430,DOUGH,TAIL,999,6


In [32]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,pair_idx,type,test,session
0,98,142,BATH,BLUEPRINT,0,-1,extra,1,0
1,1309,1083,SINK,PLANT,0,-1,extra,1,0
2,1058,-1,PICK,ZOO,0,-1,extra,1,0
3,754,-1,JELLO,ZOO,1,24,Same Item,1,0
4,1601,1409,WHALE,SUPERMARKET,1,6,Intact Pair,1,0
...,...,...,...,...,...,...,...,...,...
119995,1078,-1,PITCHFORK,ZOO,0,-1,extra,2,999
119996,139,1585,BLOUSE,WALRUS,0,-1,extra,2,999
119997,70,-1,BAIT,ZOO,0,27,Pair-Item,2,999
119998,50,1176,ASTEROID,RECORD,0,-1,extra,2,999


### Run CMR-IA

In [33]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [34]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.4,
    beta_rec = 0.1, # beta for retrieved item
    beta_cue = 0.3,
    beta_distract = 0.3,
    gamma_fc = 0.03,
    s_fc = 0.3,
    c_thresh = 0.30,
    c_thresh_ass = 0.50,
    learn_while_retrieving = True,
    d_ass = 0.3,
)
params

{'beta_enc': 0.4,
 'beta_rec': 0.1,
 'beta_cue': 0.3,
 'beta_rec_post': 0.5,
 'beta_distract': 0.3,
 'phi_s': 2,
 'phi_d': 0.5,
 's_cf': 0,
 's_fc': 0.3,
 'kappa': 0.5,
 'eta': 0.5,
 'omega': 8,
 'alpha': 1,
 'c_thresh': 0.3,
 'c_thresh_ass': 0.5,
 'd_ass': 0.3,
 'lamb': 0.5,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.03,
 'gamma_cf': 0.5}

In [35]:
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params,df_study, df_test, s_mat, mode="Recog-Recog")
df_simu

CMR2 Time: 99.36099863052368


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim
0,0,98,142,0.0,358.012238,0.197160
1,0,1309,1083,0.0,903.304077,0.243434
2,0,1058,-1,0.0,1795.571655,0.277785
3,0,754,-1,1.0,130.801971,0.453185
4,0,1601,1409,1.0,24.837231,0.536252
...,...,...,...,...,...,...
119995,999,1078,-1,1.0,71.061119,0.483692
119996,999,139,1585,0.0,1157.699341,0.344159
119997,999,70,-1,1.0,224.685257,0.426134
119998,999,50,1176,0.0,108.802109,0.137608


In [36]:
df_simu['test'] = df_test['test']
df_simu = df_simu.merge(df_test,on=['session','test','test_itemno1','test_itemno2'])

df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
def get_cond(x):
    this_type = x['type']
    target = x['correct_ans']
    if target == 1:
        if this_type == "Different Item":
            return "Different Item"
        elif this_type == "Item-Pair":
            return "Item-Pair"
        elif this_type == "Pair-Item":
            return "Pair-Item"
        elif this_type == "Same Item":
            return "Same Item"
        elif this_type == "Intact Pair":
            return "Intact Pair"
    elif target == 0:
        if this_type == "extra":
            return "NR Lure"
        elif this_type == "Same Item" or this_type == "Intact Pair":
            return "Repearted Lure"
        else:
            return "Discard"
df_simu['condition'] = df_simu.apply(get_cond,axis=1)
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,correct,condition
0,0,98,142,0.0,358.012238,0.197160,1,BATH,BLUEPRINT,0,-1,extra,True,NR Lure
1,0,1309,1083,0.0,903.304077,0.243434,1,SINK,PLANT,0,-1,extra,True,NR Lure
2,0,1058,-1,0.0,1795.571655,0.277785,1,PICK,ZOO,0,-1,extra,True,NR Lure
3,0,754,-1,1.0,130.801971,0.453185,1,JELLO,ZOO,1,24,Same Item,True,Same Item
4,0,1601,1409,1.0,24.837231,0.536252,1,WHALE,SUPERMARKET,1,6,Intact Pair,True,Intact Pair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,1078,-1,1.0,71.061119,0.483692,2,PITCHFORK,ZOO,0,-1,extra,False,NR Lure
119996,999,139,1585,0.0,1157.699341,0.344159,2,BLOUSE,WALRUS,0,-1,extra,True,NR Lure
119997,999,70,-1,1.0,224.685257,0.426134,2,BAIT,ZOO,0,27,Pair-Item,False,Discard
119998,999,50,1176,0.0,108.802109,0.137608,2,ASTEROID,RECORD,0,-1,extra,True,NR Lure


In [37]:
df_simu['pair'] = df_simu.apply(lambda x: False if x['test_itemno2'] == -1 else True, axis=1)
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,correct,condition,pair
0,0,98,142,0.0,358.012238,0.197160,1,BATH,BLUEPRINT,0,-1,extra,True,NR Lure,True
1,0,1309,1083,0.0,903.304077,0.243434,1,SINK,PLANT,0,-1,extra,True,NR Lure,True
2,0,1058,-1,0.0,1795.571655,0.277785,1,PICK,ZOO,0,-1,extra,True,NR Lure,False
3,0,754,-1,1.0,130.801971,0.453185,1,JELLO,ZOO,1,24,Same Item,True,Same Item,False
4,0,1601,1409,1.0,24.837231,0.536252,1,WHALE,SUPERMARKET,1,6,Intact Pair,True,Intact Pair,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,1078,-1,1.0,71.061119,0.483692,2,PITCHFORK,ZOO,0,-1,extra,False,NR Lure,False
119996,999,139,1585,0.0,1157.699341,0.344159,2,BLOUSE,WALRUS,0,-1,extra,True,NR Lure,True
119997,999,70,-1,1.0,224.685257,0.426134,2,BAIT,ZOO,0,27,Pair-Item,False,Discard,False
119998,999,50,1176,0.0,108.802109,0.137608,2,ASTEROID,RECORD,0,-1,extra,True,NR Lure,True


In [38]:
csim_two = df_simu.groupby(["pair","correct_ans"]).csim.mean()
csim_two

pair   correct_ans
False  0              0.358415
       1              0.454348
True   0              0.356900
       1              0.480278
Name: csim, dtype: float64

In [39]:
opt_thresh_item = np.mean(csim_two[False])
opt_thresh_pair = np.mean(csim_two[True])
print(opt_thresh_item)
print(opt_thresh_pair)
def adjust_resp(x):
    ispair = x['pair']
    csim = x['csim']
    if ispair:
        if csim > opt_thresh_pair:
            return 1
        else:
            return 0
    else:
        if csim > opt_thresh_item:
            return 1
        else:
            return 0
df_simu['s_resp'] = df_simu.apply(adjust_resp, axis=1)
df_simu

0.40638185316442477
0.41858946659266333


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,correct,condition,pair
0,0,98,142,0,358.012238,0.197160,1,BATH,BLUEPRINT,0,-1,extra,True,NR Lure,True
1,0,1309,1083,0,903.304077,0.243434,1,SINK,PLANT,0,-1,extra,True,NR Lure,True
2,0,1058,-1,0,1795.571655,0.277785,1,PICK,ZOO,0,-1,extra,True,NR Lure,False
3,0,754,-1,1,130.801971,0.453185,1,JELLO,ZOO,1,24,Same Item,True,Same Item,False
4,0,1601,1409,1,24.837231,0.536252,1,WHALE,SUPERMARKET,1,6,Intact Pair,True,Intact Pair,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,1078,-1,1,71.061119,0.483692,2,PITCHFORK,ZOO,0,-1,extra,False,NR Lure,False
119996,999,139,1585,0,1157.699341,0.344159,2,BLOUSE,WALRUS,0,-1,extra,True,NR Lure,True
119997,999,70,-1,1,224.685257,0.426134,2,BAIT,ZOO,0,27,Pair-Item,False,Discard,False
119998,999,50,1176,0,108.802109,0.137608,2,ASTEROID,RECORD,0,-1,extra,True,NR Lure,True


In [40]:
# Yes Rate
df_target = df_simu.query("condition != 'Discard'")
df_target.groupby(["test","condition"])["s_resp"].mean()

test  condition     
1     Different Item    0.707791
      Intact Pair       0.746720
      Item-Pair         0.714819
      NR Lure           0.311367
      Pair-Item         0.746448
      Repearted Lure    0.333333
      Same Item         0.713885
2     Different Item    0.698578
      Intact Pair       0.750696
      Item-Pair         0.760661
      NR Lure           0.315167
      Pair-Item         0.725035
      Repearted Lure    0.372155
      Same Item         0.757163
Name: s_resp, dtype: float64

In [41]:
def get_pair(df_tmp):
    df_tmp_pair = pd.pivot_table(df_tmp,index=["pair_idx","condition"],columns="test",values="correct")
    df_tmp_pair.columns = ["test1","test2"]
    df_tmp_pair.reset_index(inplace=True)
    return df_tmp_pair

df_p = df_target.query("condition != 'NR Lure'")
df_pair = df_p.groupby("session").apply(get_pair).reset_index()
df_pair

,session,level_1,pair_idx,condition,test1,test2
0,0,0,0,Item-Pair,1.0,1.0
1,0,1,1,Pair-Item,0.0,1.0
2,0,2,2,Same Item,1.0,1.0
3,0,3,3,Pair-Item,1.0,1.0
4,0,4,4,Intact Pair,0.0,0.0
...,...,...,...,...,...,...
26489,999,22,22,Pair-Item,0.0,1.0
26490,999,23,23,Intact Pair,0.0,0.0
26491,999,24,24,Different Item,1.0,1.0
26492,999,25,26,Repearted Lure,0.0,0.0


In [42]:
# Q value
conditions = np.unique(df_pair.condition)
for cond in conditions:
    print(cond)
    
    df_tmp = df_pair.loc[df_pair.condition == cond]
    df_tab = pd.crosstab(index=df_tmp.test2,columns=df_tmp.test1,normalize=False)
    print(df_tab)

    def Yule_Q(A, B, C, D):
        return (A * D - B * C) / (A * D + B * C)

    try:    
        print("Q: ", Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5))
    except:
        print("No Q")

Different Item
test1  0.0   1.0
test2           
0.0     37   259
1.0    271  4426
Q:  0.4040613580998782
Intact Pair
test1   0.0   1.0
test2            
0.0    1257    60
1.0     137  1061
Q:  0.9876131198558618
Item-Pair
test1  0.0   1.0
test2           
0.0    390  3627
1.0     46  3441
Q:  0.776966179410142
Pair-Item
test1   0.0   1.0
test2            
0.0     264    19
1.0    2505  2209
Q:  0.8456985172201431
Repearted Lure
test1  0.0  1.0
test2          
0.0    746   63
1.0     20  665
Q:  0.9947731116899804
Same Item
test1  0.0   1.0
test2           
0.0    215    26
1.0     79  4671
Q:  0.9958233174566199
